In [20]:

import httpx


async def fetch_models(base_url: str, api_key: str | None = None) -> dict:
    url = f"{base_url.rstrip('/')}/models"
    headers = {"Authorization": f"Bearer {api_key}"} if api_key else None
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.get(url, headers=headers)
        response.raise_for_status()
        return response.json()


def parse_model_ids(response: dict) -> list[str]:
    return [model.get("id") for model in response.get("data", []) if "id" in model]


Notes:

- reasoning: enabled 
- sometimes different pricing for +128k context


In [21]:
api_key = "xai-5cOB2MswjBKa9DEDU3gbCo02XVUUAC0nY6NYJ6Df0EYhL6PXoz2PEI9fTi4ilCUnEW9zNPbfzedrMgG9"
base_url = "https://api.x.ai/v1"
provider_id: str | None = "x-ai"

In [25]:
openrouter_models_response = await fetch_models("https://openrouter.ai/api/v1")
openrouter_models_list = openrouter_models_response.get("data", [])
print("openrouter models:", len(openrouter_models_list))

provider_models_response = await fetch_models(base_url, api_key)
provider_model_ids = parse_model_ids(provider_models_response)
print("provider models:", len(provider_model_ids))

if provider_id:
    openrouter_models_list = [model for model in openrouter_models_list if model.get("id").startswith(provider_id)]
    print("filtered models:", len(openrouter_models_list))


openrouter models: 340
provider models: 9
filtered models: 7


In [28]:
models_map: dict[str, dict | None] = {}

for model_id in provider_model_ids:
    model = next(
        (
            model
            for model in openrouter_models_list
            if (model.get("id") == model_id)
            or (model.get("id").split("/")[-1] == model_id)
            or (model.get("canonical_slug") == model_id)
            or (model.get("canonical_slug").split("/")[-1] == model_id)
        ),
        None,
    )
    if model:
        models_map[model_id] = model
    else:
        models_map[model_id] = None

print("\nFound models:")
for model_id, or_model in models_map.items():
    if or_model:
        print(model_id, or_model.get("id"))

print("\nNot  found models:")
for model_id, or_model in models_map.items():
    if not or_model:
        print(model_id)

print("\nUnmatched models:")
for model in openrouter_models_list:
    if model.get("id").split("/")[-1] not in models_map:
        print(model.get("id"), model.get("canonical_slug"))





Found models:
grok-3 x-ai/grok-3
grok-3-mini x-ai/grok-3-mini
grok-code-fast-1 x-ai/grok-code-fast-1

Not  found models:
grok-2-1212
grok-2-vision-1212
grok-4-0709
grok-4-fast-non-reasoning
grok-4-fast-reasoning
grok-2-image-1212

Unmatched models:
x-ai/grok-4-fast x-ai/grok-4-fast
x-ai/grok-4 x-ai/grok-4-07-09
x-ai/grok-3-mini-beta x-ai/grok-3-mini-beta
x-ai/grok-3-beta x-ai/grok-3-beta
